<a href="https://colab.research.google.com/github/AlessandroDiGioacchino/amd-project/blob/main/finding_similar_items.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import files

files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alessandrodigio","key":"6f89423630ec7ffb9c31bef52bcb6999"}'}

In [ ]:

!pip install -q kaggle

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d yelp-dataset/yelp-dataset

100% 4.06G/4.07G [00:34<00:00, 127MB/s]
100% 4.07G/4.07G [00:34<00:00, 127MB/s]


In [ ]:
!unzip -n yelp-dataset.zip -d yelp-dataset

Archive:  yelp-dataset.zip
  inflating: yelp-dataset/Dataset_User_Agreement.pdf  
  inflating: yelp-dataset/yelp_academic_dataset_business.json  
  inflating: yelp-dataset/yelp_academic_dataset_checkin.json  
  inflating: yelp-dataset/yelp_academic_dataset_review.json  
  inflating: yelp-dataset/yelp_academic_dataset_tip.json  
  inflating: yelp-dataset/yelp_academic_dataset_user.json  


In [ ]:

import json
import pandas as pd
import random


data_file = 'yelp-dataset/yelp_academic_dataset_review.json'

length_limit = 524288
json_lines = []

with open( data_file, 'r' ) as json_file:
  for _ in range( length_limit ):
    line = json_file.readline()

    if not line:
      break

    json_lines.append( line )


sample_size = 1024
k = 5  # k-grams length
n = 256  # Amount of signatures
t = .6  # Similarity threshold

random.seed( round( 5 * 128 * .9 ) )
random_sample = random.sample( json_lines, sample_size )
parsed_data = [ json.loads( line ) for line in random_sample ]
review_df = pd.DataFrame( parsed_data )


In [ ]:
review_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,zHPV2wVdJRCXXUiOsYR1EQ,kBKBZ3TQctQSR7rvev_E4Q,YxhdfdhRZjJHPFxaADnNVg,1.0,0,0,0,I purchased a car drove it home had a check en...,2021-12-06 21:48:01
1,CYHevvs__TxU5a1bO6Tkvw,gggWd4tfuVdjSYxYFOectw,sr-5EY6bmp4jINhea06MjA,4.0,0,0,0,Took my mom here for lunch on Friday and altho...,2018-06-15 02:27:50
2,nNs7crTxUfQO1P4EoiMYrQ,b63eLCGDCpVPtswPtyq8bA,Uc6PdjT_MO5bRPQsUUEy5Q,4.0,0,0,0,Pretty good place it only gets 4 stars because...,2016-10-01 02:19:05
3,DNmlFmzlf6EeoQAMZPh4fg,huqGHigM2ZQ7UHRFuGSkdQ,uKEabBQrn0gLzvTppOrKIA,5.0,0,0,1,I usually go to small mom and pops for tacos b...,2018-10-26 22:07:31
4,tb3zhulP4YgSuKZm3Ez0NA,lW62LGwdg-oSLTaHQ-jviQ,ORL4JE6tz3rJxVqkdKfegA,5.0,0,0,0,The opryland hotel and resort is very nice rea...,2018-10-10 18:32:11


In [ ]:
review_df[['review_id', 'text']].head()

,review_id,text
0,zHPV2wVdJRCXXUiOsYR1EQ,I purchased a car drove it home had a check en...
1,CYHevvs__TxU5a1bO6Tkvw,Took my mom here for lunch on Friday and altho...
2,nNs7crTxUfQO1P4EoiMYrQ,Pretty good place it only gets 4 stars because...
3,DNmlFmzlf6EeoQAMZPh4fg,I usually go to small mom and pops for tacos b...
4,tb3zhulP4YgSuKZm3Ez0NA,The opryland hotel and resort is very nice rea...


In [ ]:

id_txt = review_df[['review_id', 'text']]
id_txt.head()


,review_id,text
0,zHPV2wVdJRCXXUiOsYR1EQ,I purchased a car drove it home had a check en...
1,CYHevvs__TxU5a1bO6Tkvw,Took my mom here for lunch on Friday and altho...
2,nNs7crTxUfQO1P4EoiMYrQ,Pretty good place it only gets 4 stars because...
3,DNmlFmzlf6EeoQAMZPh4fg,I usually go to small mom and pops for tacos b...
4,tb3zhulP4YgSuKZm3Ez0NA,The opryland hotel and resort is very nice rea...


In [ ]:
id_txt_rows = id_txt.values.tolist()

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:

spark_version = '3.5.0'

!wget -q https://downloads.apache.org/spark/spark-{spark_version}/spark-{spark_version}-bin-hadoop3.tgz
!tar -xf spark-{spark_version}-bin-hadoop3.tgz


In [ ]:
!pip install -q findspark

In [ ]:

import os

os.environ[ 'JAVA_HOME' ] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ[ 'SPARK_HOME' ] = f'/content/spark-{spark_version}-bin-hadoop3'


In [ ]:

import findspark

findspark.init( os.environ[ 'SPARK_HOME' ] )


In [ ]:

from pyspark.sql import SparkSession

spark = SparkSession.builder.master( 'local[*]' ).getOrCreate()


In [ ]:
sc = spark.sparkContext

In [ ]:
id_txt_rdd = sc.parallelize( id_txt_rows )

In [ ]:

id_txt_lower = id_txt_rdd.map(
    lambda id_txt: ( id_txt[ 0 ], id_txt[ 1 ].lower() )
)


In [ ]:

id_words_rdd = id_txt_lower.map(
    lambda id_txt: ( id_txt[ 0 ], id_txt[ 1 ].split() )
)

force_exec = id_words_rdd.take( 1 )


In [ ]:

import nltk


def remove_stopwords( word_list: list[ str ] ) -> list[ str ]:
  from nltk.corpus import stopwords  # imported here to avoid errors later
  nltk.download( 'stopwords' )

  no_stopwords = []

  for word in word_list:
    if word not in stopwords.words( 'english' ):
      no_stopwords.append( word )

  return no_stopwords


no_stopwords_rdd = id_words_rdd.map(
    lambda id_words: ( id_words[ 0 ], remove_stopwords( id_words[ 1 ] ) )
)


In [ ]:

import string


def remove_punctuation( word_list: list[ str ] ) -> list[ str ]:
  no_punctuation = []

  for word in word_list:
    no_punctuation.append( word.translate(
        str.maketrans( '', '', string.punctuation ) )
    )

  return no_punctuation


In [ ]:

no_punctuation = no_stopwords_rdd.map(
    lambda id_words: ( id_words[ 0 ], remove_punctuation( id_words[ 1 ] ) )
)

force_exec = no_punctuation.take( 1 )


In [ ]:

no_stopwords = no_punctuation.map(
    lambda id_words: ( id_words[ 0 ], remove_stopwords( id_words[ 1 ] ) )
)


In [ ]:

import re

def is_numeric( word: str ) -> bool:
  numeric_pattern = re.compile( r'^[0-9]+$' )
  return bool( numeric_pattern.match( word ) )


no_numbers_rdd = no_stopwords.map(
    lambda id_words: (
        id_words[ 0 ],
        [ word for word in id_words[ 1 ] if not is_numeric( word ) ]
    )
)

force_exec = no_numbers_rdd.take( 1 )


In [ ]:

from nltk.stem import PorterStemmer

nltk.download( 'punkt' )
stemmer = PorterStemmer()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:

def to_stem( word_list: list[ str ] ) -> list[ str ]:
  stems = []

  for word in word_list:
    stems.append( stemmer.stem( word ) )

  return stems


stem_rdd = no_numbers_rdd.map(
    lambda id_words: ( id_words[ 0 ], to_stem( id_words[ 1 ] ) )
)


In [ ]:

def join_words( word_list: list[ str ]) -> list[ str ]:
  joined_words = ''

  for word in word_list:
    joined_words = joined_words + word
    if word != word_list[ -1 ]:
      joined_words = joined_words + ' '

  return joined_words


cleaned_rdd = stem_rdd.map(
    lambda id_txt: ( id_txt[ 0 ], join_words( id_txt[ 1 ] ) )
)

force_exec = cleaned_rdd.take( 1 )


In [ ]:

def extract_kgrams( s: str, n: int ) -> list[ str ]:
  kgrams = []

  for i in range( len( s ) - n + 1 ):
    kgram = s[ i : i + n ]
    kgrams.append( kgram )

  return kgrams


kgrams_rdd = cleaned_rdd.map(
    lambda id_txt: ( id_txt[ 0 ], extract_kgrams( id_txt[ 1 ], k ) )
)


In [ ]:

import hashlib


def create_hash_function( i, result_size=4 ):
  salt = str( i ).encode( 'utf-8' )

  def hash_string( input_string ):
    data = str( input_string ).encode( 'utf-8' ) + salt
    hashed = hashlib.sha256( data ).hexdigest()

    return hashed[ :result_size ]

  return hash_string


In [ ]:

from typing import Callable


def hash_kgrams(
    kgrams: list[ str ], hash_func: Callable[ [ str ], str ] ) -> list[ str ]:

  hashed_kgrams = []

  for kgram in kgrams:
    hashed_kgrams.append( hash_func( kgram ) )

  return hashed_kgrams


hash_func = create_hash_function( 123 )

hashed_kgrams = kgrams_rdd.map(
    lambda kgram: ( kgram[ 0 ], hash_kgrams( kgram[ 1 ], hash_func ) )
)

force_exec = hashed_kgrams.take( 1 )


In [ ]:

def min_hash( kgrams: list, k: int ) -> int:
  hash_func = create_hash_function( k )
  min_signature = float( 'inf' )

  for kgram in kgrams:
    hash_signature = int( hash_func( kgram ), 16 )

    if hash_signature < min_signature:
      min_signature = hash_signature

  return min_signature


In [ ]:

signature_matrix = hashed_kgrams.map(
    lambda h_kgram:
    ( h_kgram[ 0 ], [ min_hash( h_kgram[ 1 ], i ) for i in range( n ) ] )
)


In [ ]:

def divide_signature_matrix( id_sig: tuple[ int, list[ int ] ], bands: int,
                             rows_per_band: int ):

  doc_id = id_sig[ 0 ]
  doc_sig = id_sig[ 1 ]

  hash_func = create_hash_function( 123 )

  out = []

  for i in range( bands ):
    band_id = i
    offset = i * rows_per_band

    column = ''.join( str( n ) for n in doc_sig[ offset : offset + rows_per_band ] )
    bucket_id = hash_func( column )

    key = ( band_id, bucket_id )
    out.append(( key, doc_id ))

  return out


In [ ]:

from scipy.optimize import fsolve


def func( x ):
  b, r = x
  eq1 = ( 1 / b )**( 1 / r ) - t
  eq2 = b * r - n

  return [ eq1, eq2 ]


solutions = fsolve( func, [ 1, 1 ] )

b_solution, r_solution = solutions


In [ ]:
( b_solution, r_solution )

(36.3845585095641, 7.035951801771666)

In [ ]:

from math import floor

# Some hashes are going to be discarded
b = floor( b_solution )
r = floor( r_solution )


In [ ]:

hashed_bands = signature_matrix.flatMap(
    lambda x: divide_signature_matrix( x, b, r )
)

force_exec = hashed_bands.take( 1 )


In [ ]:

candidates = hashed_bands.groupByKey().map(
    lambda x: ( x[ 0 ], list( x[ 1 ] ) )
)


In [ ]:

filtered_candidates = candidates.filter(
    lambda collision_list: len( collision_list[ 1 ] ) > 1
)

force_exec = filtered_candidates.take( 1 )


In [ ]:

import itertools


def get_pairs( collisions_list: list ) -> tuple:
  pair_list = []

  for pair in itertools.combinations( collisions_list, 2 ):
    pair_list.append( pair )

  return tuple( pair_list )


In [ ]:

candidate_pairs = filtered_candidates.flatMap(
    lambda pairs: get_pairs( pairs[ 1 ] )
).distinct()


In [ ]:

candidate_pairs_set = candidate_pairs.collect()
candidate_reviews = { 'id1': [], 'txt1': [], 'id2': [], 'txt2': [] }

for candidate_pair in candidate_pairs_set:
  id1, id2 = candidate_pair
  txt1 = id_txt[ id_txt.review_id == id1 ].text.values[ 0 ]
  txt2 = id_txt[ id_txt.review_id == id2 ].text.values[ 0 ]

  candidate_reviews[ 'id1' ].append( id1 )
  candidate_reviews[ 'txt1' ].append( txt1 )
  candidate_reviews[ 'id2' ].append( id2 )
  candidate_reviews[ 'txt2' ].append( txt2 )

candidate_reviews_df = pd.DataFrame.from_dict( candidate_reviews )


In [ ]:

candidates_set = { candidate for candidate_pair in candidate_pairs_set
                             for candidate in candidate_pair }


In [ ]:

# candidate_kgrams = kgrams_rdd.filter(
#     lambda id_kgrams: id_kgrams[ 0 ] in candidates_set
# )

candidate_signatures = signature_matrix.filter(
    lambda id_signature: id_signature[ 0 ] in candidates_set
)


In [ ]:

# cartesian_kgrams = candidate_kgrams.cartesian( candidate_kgrams )
cartesian_signatures = candidate_signatures.cartesian( candidate_signatures )


In [ ]:

# filtered_cartesian_kgrams = cartesian_kgrams.filter(
#     lambda el: ( el[ 0 ][ 0 ], el[ 1 ][ 0 ] ) in candidate_pairs_set
# )

filtered_cartesian_signatures = cartesian_signatures.filter(
    lambda el: ( el[ 0 ][ 0 ], el[ 1 ][ 0 ] ) in candidate_pairs_set
)


In [ ]:

def jaccard_sim( set1: set, set2: set ) -> float:
  intersection_length = len( set1.intersection( set2 ) )
  union_length = len( set1.union( set2 ) )

  return float( intersection_length ) / float( union_length )


In [ ]:

# jaccard_kgrams_rdd = filtered_cartesian_kgrams.map(
#     lambda el: ( el[ 0 ][ 0 ], el[ 1 ][ 0 ],
#                  jaccard_sim( set( el[ 0 ][ 1 ] ), set( el[ 1 ][ 1 ] ) ) )
# )

# jaccard_kgrams_df = jaccard_kgrams_rdd.toDF()
# jaccard_kgrams_df.head()


In [ ]:

# jaccard_kgrams_pd = jaccard_kgrams_df.toPandas()
# jaccard_kgrams_pd = jaccard_kgrams_pd.sort_values( by=[ '_3' ], ascending=False )
# jaccard_kgrams_pd.head( 4 )


In [ ]:

jaccard_signatures_rdd = filtered_cartesian_signatures.map(
    lambda el: ( el[ 0 ][ 0 ], el[ 1 ][ 0 ],
                 jaccard_sim( set( el[ 0 ][ 1 ] ), set( el[ 1 ][ 1 ] ) ) )
)

force_exec = jaccard_signatures_rdd.take( 1 )
# jaccard_signatures_df = jaccard_signatures_rdd.toDF()
# force_exec = jaccard_signatures_df.take( 1 )


In [ ]:

# jaccard_signatures_pd = jaccard_signatures_df.toPandas()
top_elements = jaccard_signatures_rdd.top( 5, key=lambda x: x[ 2 ] )
print( top_elements )


[('o5O3CYV79dTN8O9mFijmFA', 'SbeoXuUHy1xgL-o7Urpkeg', 0.4406779661016949), ('rF18WhkhtFysLHc8zX2ypA', '6eYJwilLx5q6FRU31i7eLA', 0.43646408839779005), ('xSl2-eyZt0Cbo_lynugdlg', 'lauwOnBdscDiWd2bxAbMBw', 0.4349775784753363), ('kRN7_ajyY-PR19x-8Mc-Hw', 'aRe94o2uD4aY-eqbAl-nLg', 0.43315508021390375), ('hojoM8hgdzF4iOc6_XZ9tQ', 'mb4jlH040AXxoOwlGdPoCQ', 0.42995169082125606)]


In [ ]:

id_txt[ id_txt.review_id == top_elements[ 0 ][ 0 ] ].text.values


array(["Finally went to Delicia! My boyfriend and I went there for date night and had a fantastic time! At 7:15 on a Tuesday, the place was lively but there was no wait for a table for 2. The decor is cool and romantic, the host was very gracious, and our waiter, Shawn, was friendly and made good recommendations. I loved my margarita, and my boyfriend's sangria was tasty - nice and winey! I also tried the mojito. It was good... but tasted like Double Mint gum to me - not in a bad way but not in a way I liked. If you tasted the same thing, please, PLEASE message me so I feel less crazy! \n\nAnyway, the queso fundido appetizer was delicious and unique (the chips were great too). For dinner, he got the beef empanadas and I got the duck enchiladas. When we go back to Delicia again (and we will), I want to try the scallops, but the duck enchiladas, rice and beans were so good it's hard to imagine ordering anything else! My boyfriend really enjoyed the flaky empanadas, as well. We chose wise

In [ ]:
id_txt[ id_txt.review_id == top_elements[ 0 ][ 1 ] ].text.values

array(['"Top donuts in the country? "\n\nOkay. Whenever I read something that says "top ten blah blah of something in the us or state of" my wife and I are extremely hesitant because it sounds like a gimmick. After coming to experience this place first hand I can understand why you would make that statement. We visited on a Thursday 10:30amish with a line of 5 people. They don\'t serve sliders until 11am and this gave us a great opportunity to do both separately.\n\nAmbience:\nAlot of character and with a ton of regulars. They were on a first name basis with half the people in the place. Located on magazine st in the garden district. Artsy stuff. Nice street to walk, explore, and shop.\n\nFood:\nThey categorize their donuts into 3 categories to simplify pricing. Reg/fancy/more fancy. This is brilliant as it speeds up the ordering but they still carry variety on their menu.\n\nGlazed donut 4/5\nThis was a good, well balanced donut. Not overly sweet but had characteristics of what you ex